In [1]:
" Starte Programm mit Run "
# For reading the Input data from Excel-File choose Input_File = True
# For reading the Input data from script "alternative_input.py" choose Input_File = False
Input_File = True

import time
start_timer = time.perf_counter()
def print_start(name):
    print(f'Start Programm {name}')
if __name__ == '__main__':
    print_start('Gesamtkonzept Strom')

%run ./Module/Simulation_Komponenten.ipynb

end_timer = time.perf_counter()
running_time_min = round((end_timer-start_timer)/60,2)
print("Programm Laufzeit = "+str(running_time_min)+" Min")
quit()

Start Simulation Gesamtkonzept Strom
Lastgang lesen und PV Eigenverbrauch berechnen
Simulation Running
Simulation ELKW
Fahrdaten einlesen
Analyse für monatliche Leistungs Limite Peak-Shaving
Simulation WELKW
Fahrdaten einlesen
Analyse für monatliche Leistungs Limite Peak-Shaving
Simulation EPKW Ladestationen
Bilanzierung PV-Anlage
Simulation Batteriespeicher
Analyse für monatliche Leistungs Limite Peak-Shaving
Simulation der Komponenten fertig!
Excel mit Resultaten erstellen
Diagramme erstellen
Programm Laufzeit = 7.24 Min
